In [1]:
import pandas as pd
import random

In [2]:
import ctypes
from ctypes import POINTER
cisin = ctypes.CDLL("./libcisin.so")
cisin.c_isin.argtypes = [POINTER(ctypes.c_char), POINTER(ctypes.c_char_p)]


In [3]:
def call_c_isn(_string: str, _list: list):
    input_phrase = _string.encode('utf-8')
    list_of_phrases = (ctypes.c_char_p * len(_list))()
    list_of_phrases[:] = [i.encode('utf-8') for i in _list]
    return cisin.c_isin(input_phrase, list_of_phrases, len(_list))

In [4]:
call_c_isn('this string', ['this string', 'another string'])

1

In [5]:
test_df = pd.DataFrame(
    {
        'description': [f'description {i}' for i in range(10)] + [f'description {i}' for i in range(10)]
    }
)

In [6]:
set_of_descriptions = [
    'description 0',
    'description 1',
    'description 2',
    'description 6',
    'description 9'
]

In [26]:
import timeit
import numpy as np

setup_1 = """
import pandas as pd
test_df = pd.DataFrame(
        {
            'description': [f'description {i}' for i in range(100)] + [f'description {i}' for i in range(100)]
        }
)

set_of_descriptions = [
    'description 0',
    'description 1',
    'description 2',
    'description 6',
    'description 99'
]
"""
setup_2 = """
import pandas as pd
import ctypes
import numpy as np
from ctypes import POINTER
cisin = ctypes.CDLL("./libcisin.so")
cisin.c_isin.argtypes = [POINTER(ctypes.c_char), POINTER(POINTER(ctypes.c_char))]


test_df = pd.DataFrame(
        {
            'description': [f'description {i}' for i in range(100)] + [f'description {i}' for i in range(100)]
        }
)
test_df['description_encoded'] = test_df['description'].apply(lambda x: x.encode('utf-8'))

set_of_descriptions = [
    'description 0',
    'description 1',
    'description 2',
    'description 6',
    'description 99'
]
set_of_descriptions_encoded = [i.encode('utf-8') for i in set_of_descriptions]

def call_c_isn(_string: str, _list: list):
    input_phrase = _string
    list_of_phrases = (ctypes.c_char_p * len(_list))()
    list_of_phrases[:] = _list
    return cisin.c_isin(input_phrase, list_of_phrases, len(_list))

"""

test_df = pd.DataFrame(
        {
            'description': [f'description {i}' for i in range(100)] + [f'description {i}' for i in range(100)]
        }
)
test_df['description_encoded'] = test_df['description'].apply(lambda x: x.encode('utf-8'))

set_of_descriptions = [
    'description 0',
    'description 1',
    'description 2',
    'description 6',
    'description 99'
]
set_of_descriptions_encoded = [i.encode('utf-8') for i in set_of_descriptions]

def call_c_isn(_string: str, _list: list):
    input_phrase = _string
    list_of_phrases = (ctypes.c_char_p * len(_list))()
    list_of_phrases[:] = _list
    return cisin.c_isin(input_phrase, list_of_phrases, len(_list))

def f_1(
    test_df=test_df,
    set_of_descriptions = set_of_descriptions
):
    test_df['filtered_description'] = 0
    test_df.loc[
        test_df['description'].isin(set_of_descriptions),
        'filtered_description'
    ] = 1
    test_df['filtered_description'] = test_df['filtered_description'].astype(bool)
    return test_df

def f_2(
    test_df=test_df,
    set_of_descriptions: list = set_of_descriptions_encoded
):
    desc = test_df['description_encoded'].values
    test_df['filtered_description'] = [call_c_isn(i, set_of_descriptions) for i in desc]
    test_df['filtered_description'] = test_df['filtered_description'].astype(bool)
    return test_df




In [27]:
f_1()

,description,description_encoded,filtered_description
0,description 0,b'description 0',True
1,description 1,b'description 1',True
2,description 2,b'description 2',True
3,description 3,b'description 3',False
4,description 4,b'description 4',False
...,...,...,...
195,description 95,b'description 95',False
196,description 96,b'description 96',False
197,description 97,b'description 97',False
198,description 98,b'description 98',False


In [28]:
f_2()

,description,description_encoded,filtered_description
0,description 0,b'description 0',True
1,description 1,b'description 1',True
2,description 2,b'description 2',True
3,description 3,b'description 3',False
4,description 4,b'description 4',False
...,...,...,...
195,description 95,b'description 95',False
196,description 96,b'description 96',False
197,description 97,b'description 97',False
198,description 98,b'description 98',False


In [29]:
timeit.timeit(f_1, setup=setup_1, number=10000)


11.533101502998761

In [30]:
timeit.timeit(f_2, setup=setup_2, number=10000)


24.358356540000386